Pre-processing

In [3]:
# Importation des librairies
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

# Importation du module os pour interagir avec le système d'exploitation
import os
path = '../../../../../images/image_train'

list_img = os.listdir(path)

print('le nombre total d\'image dans le repertoire image_train est de :', len(list_img))


le nombre total d'image dans le repertoire image_train est de : 84916


In [4]:
# Chargement des fichiers '="X_train_uptade.csv" et "Y_trainCVw08PX.csv"
X_train = pd.read_csv('../../../../../X_train.csv', index_col=0)
y_train = pd.read_csv('../../../../../y_train.csv', index_col=0)

# Fusion avec merge des deux datasets
df = pd.merge(X_train, y_train, left_index = True, right_index = True)

# Affichage des 5 premières lignes
display(df.head())

# Affichage des 5 dernières lignes
display(df.tail())

,designation,description,productid,imageid,prdtypecode
0,Olivia: Personalisiertes Notizbuch / 150 Seite...,NaN,3804725264,1263597046,10
1,Journal Des Arts (Le) N° 133 Du 28/09/2001 - L...,NaN,436067568,1008141237,2280
2,Grand Stylet Ergonomique Bleu Gamepad Nintendo...,PILOT STYLE Touch Pen de marque Speedlink est ...,201115110,938777978,50
3,Peluche Donald - Europe - Disneyland 2000 (Mar...,NaN,50418756,457047496,1280
4,La Guerre Des Tuques,Luc a des id&eacute;es de grandeur. Il veut or...,278535884,1077757786,2705


,designation,description,productid,imageid,prdtypecode
84911,The Sims [ Import Anglais ],NaN,206719094,941495734,40
84912,Kit piscine acier NEVADA déco pierre Ø 3.50m x...,<b>Description complète :</b><br />Kit piscine...,3065095706,1188462883,2583
84913,Journal Officiel De La Republique Francaise N°...,NaN,440707564,1009325617,2280
84914,Table Basse Bois De Récupération Massif Base B...,<p>Cette table basse a un design unique et con...,3942400296,1267353403,1560
84915,Gomme De Collection 2 Gommes Pinguin Glace Ver...,NaN,57203227,684671297,2522


In [5]:
# création d'une nouvelle colonne
df['Nom image'] = ['image_' + str(imageid) + '_product_' + str(productid) + '.jpg' for imageid, productid in zip(df['imageid'], df['productid'])]
df['lien'] = str(path) + '/' + df['Nom image']
display(df.head())

df.info()

,designation,description,productid,imageid,prdtypecode,Nom image,lien
0,Olivia: Personalisiertes Notizbuch / 150 Seite...,NaN,3804725264,1263597046,10,image_1263597046_product_3804725264.jpg,../../../../../images/image_train/image_126359...
1,Journal Des Arts (Le) N° 133 Du 28/09/2001 - L...,NaN,436067568,1008141237,2280,image_1008141237_product_436067568.jpg,../../../../../images/image_train/image_100814...
2,Grand Stylet Ergonomique Bleu Gamepad Nintendo...,PILOT STYLE Touch Pen de marque Speedlink est ...,201115110,938777978,50,image_938777978_product_201115110.jpg,../../../../../images/image_train/image_938777...
3,Peluche Donald - Europe - Disneyland 2000 (Mar...,NaN,50418756,457047496,1280,image_457047496_product_50418756.jpg,../../../../../images/image_train/image_457047...
4,La Guerre Des Tuques,Luc a des id&eacute;es de grandeur. Il veut or...,278535884,1077757786,2705,image_1077757786_product_278535884.jpg,../../../../../images/image_train/image_107775...


<class 'pandas.core.frame.DataFrame'>
Index: 84916 entries, 0 to 84915
Data columns (total 7 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   designation  84916 non-null  object
 1   description  55116 non-null  object
 2   productid    84916 non-null  int64 
 3   imageid      84916 non-null  int64 
 4   prdtypecode  84916 non-null  int64 
 5   Nom image    84916 non-null  object
 6   lien         84916 non-null  object
dtypes: int64(3), object(4)
memory usage: 7.2+ MB


In [7]:
# Sauvegarder le DataFrame df
df.to_csv('dataframe_before_random_forest_images.csv', index=False)

Préparation pour le modèle Random Forest

In [ ]:
# Recharger le DataFrame df
df = pd.read_csv('dataframe_before_random_forest_images.csv')

In [5]:

from skimage.io import imread
from skimage.transform import resize
from skimage.color import rgb2gray

# Fonction pour lire et prétraiter une image
def preprocess_image(image_path, target_size=(64, 64)):
    image = imread(image_path)
    image = rgb2gray(image)  # Convertir en noir et blanc
    image = resize(image, target_size)  # Redimensionner l'image
    return image.flatten()  # Aplatir l'image en un vecteur 1D

# Appliquer le prétraitement à toutes les images et stocker les caractéristiques
features = []
for image_lien in df['lien']:
    features.append(preprocess_image(image_lien))

features = np.array(features)

# Préparer les données pour l'entraînement
X = features
y = df['prdtypecode'].values





In [6]:
# Sauvegarder les caractéristiques X et les étiquettes y
np.save('features_before_random_forest_images.npy', X)
np.save('labels_before_random_forest_images.npy', y)

Entrainement du randomForest

In [6]:
# Recharger les caractéristiques X et les étiquettes y
X = np.load('features_before_random_forest_images.npy')
y = np.load('labels_before_random_forest_images.npy')

In [8]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split

# Diviser les données en ensembles d'entraînement et de test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Entraîner un modèle Random Forest
clf = RandomForestClassifier(n_estimators=100, random_state=42, n_jobs=-1)
clf.fit(X_train, y_train)

RandomForestClassifier(n_jobs=-1, random_state=42)

Evaluation du random forest

In [9]:
from sklearn.metrics import accuracy_score

# Évaluer le modèle
y_pred = clf.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print(f'Accuracy: {accuracy*100:.2f}%')

Accuracy: 47.52%


##### GridSearch

In [7]:
from sklearn.model_selection import GridSearchCV

# Diviser les données en ensembles d'entraînement et de test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Configuration de GridSearchCV
param_grid = {
    'n_estimators': [100, 150, 200],
    'max_depth': [None, 10, 20],
    'min_samples_split': [2, 5, 7],
    'min_samples_leaf': [1, 2, 3]
}

# Initialiser le modèle Random Forest
rf = RandomForestClassifier(random_state=42, n_jobs=-1)

# Initialiser GridSearchCV
grid_search = GridSearchCV(estimator=rf, param_grid=param_grid, cv=3, n_jobs=-1, verbose=2, scoring='accuracy')

# Exécuter GridSearchCV sur les données d'entraînement
grid_search.fit(X_train, y_train)

# Afficher les meilleurs paramètres
print("Meilleurs paramètres:", grid_search.best_params_)

# Utiliser le meilleur modèle trouvé par GridSearchCV
best_model = grid_search.best_estimator_

# Évaluer le modèle
y_pred = best_model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print(f'Accuracy du meilleur modèle: {accuracy*100:.2f}%')


Fitting 3 folds for each of 81 candidates, totalling 243 fits


In [1]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.metrics import accuracy_score

# Diviser les données en ensembles d'entraînement et de test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Configuration pour RandomizedSearchCV
param_distributions = {
    'n_estimators': [100, 200, 300],
    'max_depth': [None, 10, 20, 30],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4]
}

# Initialiser le modèle Random Forest
rf = RandomForestClassifier(random_state=42, n_jobs=-1)

# Initialiser RandomizedSearchCV
random_search = RandomizedSearchCV(estimator=rf, param_distributions=param_distributions, n_iter=10, cv=2, n_jobs=-1, verbose=2, scoring='accuracy')

# Exécuter RandomizedSearchCV sur les données d'entraînement
random_search.fit(X_train, y_train)

# Afficher les meilleurs paramètres
print("Meilleurs paramètres:", random_search.best_params_)

# Utiliser le meilleur modèle trouvé par RandomizedSearchCV
best_model = random_search.best_estimator_

# Évaluer le modèle
y_pred = best_model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print(f'Accuracy du meilleur modèle: {accuracy*100:.2f}%')



NameError: name 'X' is not defined

In [ ]:
import joblib  # Pour sauvegarder le modèle

# Sauvegarder le meilleur modèle
joblib.dump(best_model, 'random_forest_best_model.joblib')